# MedInforma API Interface

This notebook provides an example of how interact with the MedInforma API,
allowing users to ask spanish medical-related questions and receive answers from a MedLine dataset.

### API configuration

In [1]:
import json
import requests
from typing import Dict, Optional

In this case, the API is running locally. So that's why the API URL is `http://localhost:8000/ask`. However, if you want to use the API in a production environment, you should change the API URL to the one you want to use.

Headers are used to specify the content type of the request. In this case, the content type is `application/json`.

In [2]:
API_URL = "http://localhost:8000/ask"
HEADERS = {"Content-Type": "application/json"}
SEPARATOR = "=" * 50

To get the answer from the MedInforma API, we need to send a POST request to the API URL with the question as a JSON payload. The output of the API will be a JSON object.


In [3]:
def query_med_informa_api(question: str) -> Optional[Dict]:
    payload = {"text": question}
    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload)
        response.raise_for_status()
        return response.json()['answer']
    except requests.RequestException as e:
        print(f"Error querying the API: {e}")
        return None

The response from the API will be a JSON object with the next form:

```json
{
    "status": "", //Could be "Success" or any other error message. Is a string.
    "query": "", //Is the question that was asked. Is a string.
    "context": [], //Is the context that was retrieved from the MedLine dataset. Is an array.
    "metadata": [], //Is the information about the retrieved context. Is an array.
    "similarity": 1, //Is the similarity between the question and the retrieved context. Is a float.
    "completion": "", //Is the answer generated by the model. Could be a string or None, depending if the model was able to generate an answer.
}
```

With this in mind, we can create a function to clear the output of the API response and display the answer generated by the model, and the sources from the API response.


In [4]:
def display_answer(question: str, api_response: Dict) -> None:
    print(SEPARATOR)
    print(f"Question: {question}")
    print("Answer:")
    print(api_response['completion'])
    print("Sources:")
    for source in api_response['metadata']:
        print(source['url'])

### Ask to MedInforma

Main function to run the MedInforma query interface.

In [8]:
print("Welcome to the MedInforma Query Interface")
question = input("Please enter your medical question: ")
    
response = query_med_informa_api(question)
    
if response:
    display_answer(question, response)
else:
    print("No answer found. Please try again with different words.")

Welcome to the MedInforma Query Interface
Question: ¿Cuál es la diferencia entre covid-19 y gripe?
Answer:
La gripe y el COVID-19 son enfermedades respiratorias causadas por virus, pero hay diferencias clave entre ellas. El COVID-19 es causado por el virus SARS-CoV-2, mientras que la gripe es causada por el virus de la influenza. 

El COVID-19 puede causar síntomas más graves que la gripe, como dificultad para respirar, neumonía y falla orgánica. La gripe suele ser más leve y se resuelve por sí sola en unos días. 

El COVID-19 se propaga más fácilmente que la gripe y tiene un período de incubación más largo. 

Hay vacunas para ambos, pero la vacuna contra la gripe es más efectiva para prevenir la enfermedad. 

Sources:
https://medlineplus.gov/spanish/flu.html
https://medlineplus.gov/spanish/covid19coronavirusdisease2019.html
